In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('commands.json').read()
intents = json.loads(data_file)


In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



In [4]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yas88\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [7]:
# Training Data Creation
training = []
# Define Output Array
output_empty = [0] * len(classes)
# Training data as Bag of words for respective sentence
for doc in documents:
    # Bag of words Intialization
    bag = []
    # tokenized word list as pattern
    pattern_words = doc[0]
    # lemmatize the wordsand define the meaning
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Creation of bag of words array with 1, if it matches in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for each pattern
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# Conversion to  np.array
random.shuffle(training)
training = np.array(training)
# create train and test dataset
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\Users\yas88\AppData\Local\Temp/ipykernel_11760/2296428912.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [8]:
# Create model - 3 layers neural network
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
10/10 [==============================] - 0s 797us/step - loss: 2.2051 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 498us/step - loss: 2.1648 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 698us/step - loss: 2.0463 - accuracy: 0.2766
Epoch 4/200
10/10 [==============================] - 0s 598us/step - loss: 1.9154 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 797us/step - loss: 1.8305 - accuracy: 0.5106
Epoch 6/200
10/10 [==============================] - 0s 596us/step - loss: 1.7476 - accuracy: 0.4255
Epoch 7/200
10/10 [==============================] - 0s 598us/step - loss: 1.6592 - accuracy: 0.5532
Epoch 8/200
10/10 [==============================] - 0s 598us/step - loss: 1.4722 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 598us/step - loss: 1.4225 - accuracy: 0.6170
Epoch 10/200
10/10 [==============================] - 0s 689us/step - loss: 1.2972 - accura

10/10 [==============================] - 0s 698us/step - loss: 0.0418 - accuracy: 1.0000
Epoch 82/200
10/10 [==============================] - 0s 598us/step - loss: 0.0159 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 598us/step - loss: 0.0698 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 498us/step - loss: 0.0470 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 598us/step - loss: 0.0496 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 598us/step - loss: 0.0425 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 598us/step - loss: 0.2951 - accuracy: 0.9574
Epoch 88/200
10/10 [==============================] - 0s 598us/step - loss: 0.0436 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 498us/step - loss: 0.0543 - accuracy: 0.9574
Epoch 90/200
10/10 [==============================] - 0s 598us/step - loss: 0.0350 - accuracy: 

10/10 [==============================] - 0s 698us/step - loss: 0.0443 - accuracy: 0.9787
Epoch 161/200
10/10 [==============================] - 0s 698us/step - loss: 0.0826 - accuracy: 0.9574
Epoch 162/200
10/10 [==============================] - 0s 498us/step - loss: 0.0775 - accuracy: 0.9574
Epoch 163/200
10/10 [==============================] - 0s 598us/step - loss: 0.0121 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 698us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 797us/step - loss: 0.0528 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 598us/step - loss: 0.0078 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 598us/step - loss: 0.0255 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 598us/step - loss: 0.0180 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 797us/step - loss: 0.0110 - a